In [25]:
!pwd

/notebooks/notebooks/SnowparkRnD/UDFs


In [26]:
!ls

RunExperiment_Sample_On_Snowpark-Copy1.ipynb  UDF_run1.ipynb  run_experiment.py
RunExperiment_Sample_On_Snowpark.ipynb	      __pycache__


In [5]:
!jupyter nbconvert RunExperiment_Sample_On_Snowpark.ipynb --to python --output run_experiment.py

[NbConvertApp] Converting notebook RunExperiment_Sample_On_Snowpark.ipynb to python
[NbConvertApp] Writing 4775 bytes to run_experiment.py


In [9]:
from run_experiment import run_exp

In [2]:
import os
from snowflake.snowpark import Session
from snowflake.snowpark.functions import udf
import pandas as pd

In [3]:
connection_parameters = {
    "account": "ug94937.us-east4.gcp",
    "user": "ADITYASINGH",
    "password": os.environ.get('SF_Password'),
    "role": "ADITYASINGH",  # optional
    "warehouse": "FOSFOR_INSIGHT_WH",  # optional
    "database": "FIRST_DB",  # optional
    "schema": "PUBLIC",  # optional
    }
session = Session.builder.configs(connection_parameters).create()
session.sql_simplifier_enabled = True

In [4]:
session.add_import("run_experiment.py", import_path="run_experiment")
session.add_packages("numpy", "pandas")

In [5]:
@udf
def udf_run_exp(sf_pass: str, dataset: str, target: str) -> pd.DataFrame:
    return run_exp(sf_pass, dataset, target)

In [32]:
algos=['snowflake.ml.modeling.naive_bayes.GaussianNB',
       'snowflake.ml.modeling.neighbors.KNeighborsClassifier',
      ]

In [8]:
session.sql(f"select udf_run_exp('Enlightme#2024','EMPLOYEE', 'LEAVEORNOT') as run").collect()

SnowparkSessionException: (1404): Cannot perform this operation because the session has been closed.

In [ ]:
@udf
def add_udf(x: int, y: int) -> int:
    return x + y
df = session.create_dataframe([[1, 2], [3, 4]], schema=["x", "y"])
df.select(add_udf("x", "y")).to_df("add_result").collect()

In [7]:
session.close()